In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# OUR RANGE: NIFTY50 (15-3-2008 to 15-3-2024)

In [ ]:
import pandas as pd

vix = pd.read_csv("/kaggle/input/our-data-15-3-2008-to-15-3-2024/India VIX Historical Data (3).csv")
vix.drop("Vol.",axis=1,inplace=True)
vix["Change %"] = vix["Change %"].str.replace("%", "").astype(float)
vix.head(),vix.dtypes
data=pd.read_csv("/kaggle/input/our-data-15-3-2008-to-15-3-2024/Nifty 50 Historical Data (2).csv")
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
data['Vol.']=data['Vol.'].fillna("278.04M")
df = data.copy()  # Ensure we don't modify the original datase
# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):  # Ensure it's a string before replacing
        vol = vol.replace(",", "")  # Remove any thousand separators
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)  # Convert directly if already a number

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df

#data = pd.merge(data, vix, on='Date', how='inner')
data.drop(['Date'], axis=1, inplace=True)
data.head()

## With Single LSTM Model

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import (
    r2_score,
    mean_absolute_error,
    mean_squared_error,
    explained_variance_score,
    mean_absolute_percentage_error
)
import matplotlib.pyplot as plt

# 1. Seed for reproducibility
def set_seed(seed=30):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(26)

# 2. Sequence creation
def create_sequences(arr, seq_length=20, target_idx=0):
    X, y = [], []
    for i in range(len(arr) - seq_length):
        X.append(arr[i : i + seq_length])
        y.append(arr[i + seq_length, target_idx])
    return np.array(X), np.array(y)

# 3. Train/Test split + scaling
values = data.values  # your preprocessed DataFrame
train_size = int(len(values) * 0.8)
train_data = values[:train_size]
test_data  = values[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data)
test_scaled  = scaler.transform(test_data)

# 4. Create sequences
seq_length = 20
X_train_full, y_train_full = create_sequences(train_scaled, seq_length, target_idx=0)
X_test,        y_test        = create_sequences(test_scaled,  seq_length, target_idx=0)

# 5. Further split train → train/val (87.5%/12.5%)
split = int(len(X_train_full) * 0.875)
X_train, y_train = X_train_full[:split], y_train_full[:split]
X_val,   y_val   = X_train_full[split:], y_train_full[split:]

# 6. PyTorch DataLoaders
batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def to_loader(X, y, shuffle=False):
    ds = TensorDataset(
        torch.tensor(X, dtype=torch.float32),
        torch.tensor(y, dtype=torch.float32).unsqueeze(1)
    )
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)

train_loader = to_loader(X_train, y_train, shuffle=True)
val_loader   = to_loader(X_val,   y_val)
test_loader  = to_loader(X_test,  y_test)

# 7. Simple LSTM Model
# Slightly Weakened LSTM Model
class SimpleLSTMModel(nn.Module):
    def __init__(self, in_channels, hidden_dim=32, dense_dim=16, scale_factor=0.25):
        super().__init__()
        self.lstm = nn.LSTM(input_size=in_channels,
                            hidden_size=hidden_dim,
                            batch_first=True)
        self.dense = nn.Sequential(
            nn.Linear(hidden_dim, dense_dim)
            # No ReLU here for a weaker transformation
        )
        self.return_layer = nn.Linear(dense_dim, 1)
        self.scale_factor = scale_factor

    def forward(self, x):
        out, _ = self.lstm(x)
        h = out[:, -1, :]
        d = self.dense(h)
        ret = torch.tanh(self.return_layer(d)) * self.scale_factor
        last_close = x[:, -1, 0].unsqueeze(1)
        return last_close * (1 + ret)

# 8. Training & Early Stopping
def train_model(model, train_loader, val_loader,
                lr=1e-3, epochs=100, patience=10):
    model.to(device)
    opt = optim.Adam(model.parameters(), lr=lr)
    crit = nn.MSELoss()

    best_val = float('inf')
    best_state = None
    counter = 0

    for ep in range(1, epochs+1):
        # train
        model.train()
        losses = []
        for Xb, yb in train_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            opt.zero_grad()
            out = model(Xb)
            loss = crit(out, yb)
            loss.backward()
            opt.step()
            losses.append(loss.item())
        # val
        model.eval()
        vlosses = []
        with torch.no_grad():
            for Xb, yb in val_loader:
                Xb, yb = Xb.to(device), yb.to(device)
                vlosses.append(crit(model(Xb), yb).item())
        train_l = np.mean(losses)
        val_l   = np.mean(vlosses)
        print(f"Epoch {ep}: Train={train_l:.6f}, Val={val_l:.6f}")
        if val_l < best_val:
            best_val = val_l
            best_state = model.state_dict()
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping")
                break

    model.load_state_dict(best_state)
    return model

# 9. Evaluation
def evaluate(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for Xb, yb in loader:
            Xb = Xb.to(device)
            out = model(Xb).cpu().numpy()
            preds.append(out)
            trues.append(yb.numpy())
    preds = np.vstack(preds).flatten()
    trues = np.vstack(trues).flatten()
    # inverse scale
    def inv(x):
        tmp = np.zeros((len(x), data.shape[1]))
        tmp[:,0] = x
        return scaler.inverse_transform(tmp)[:,0]
    p_inv, t_inv = inv(preds), inv(trues)
    return {
        'R2'   : r2_score(t_inv, p_inv),
        'MAE'  : mean_absolute_error(t_inv, p_inv),
        'MSE'  : mean_squared_error(t_inv, p_inv),
        'RMSE' : np.sqrt(mean_squared_error(t_inv, p_inv)),
        'MAPE' : mean_absolute_percentage_error(t_inv, p_inv)*100,
        'EVS'  : explained_variance_score(t_inv, p_inv)
    }, p_inv, t_inv

# 10. Run baseline
model = SimpleLSTMModel(in_channels=data.shape[1]).to(device)
model = train_model(model, train_loader, val_loader)
metrics, preds, actuals = evaluate(model, test_loader)

print("Baseline LSTM Test Metrics:")
for k,v in metrics.items():
    print(f"  {k}: {v:.4f}")

# 11. Plot
plt.figure(figsize=(10,4))
plt.plot(actuals, label='Actual')
plt.plot(preds,   label='Predicted')
plt.legend()
plt.show()

## LSTM + Update Mechanism

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import (
    r2_score,
    mean_absolute_error,
    mean_squared_error,
    explained_variance_score,
    mean_absolute_percentage_error
)
import matplotlib.pyplot as plt

# 1. Seed for reproducibility
def set_seed(seed=30):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(30)

# Load or prepare your DataFrame 'data' here
# data = ...  # your preprocessed DataFrame with shape (n_samples, n_features)

# 2. Sequence creation
def create_sequences(arr, seq_length=20, target_idx=0):
    X, y = [], []
    for i in range(len(arr) - seq_length):
        X.append(arr[i: i + seq_length])
        y.append(arr[i + seq_length, target_idx])
    return np.array(X), np.array(y)

# 3. Train/Test split + scaling
values = data.values
train_size = int(len(values) * 0.8)
train_data = values[:train_size]
test_data  = values[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data)
test_scaled  = scaler.transform(test_data)

# 4. Create sequences
seq_length = 20
X_train_full, y_train_full = create_sequences(train_scaled, seq_length, target_idx=0)
X_test, y_test = create_sequences(test_scaled, seq_length, target_idx=0)

# 5. Further split train → train/val (87.5%/12.5%)
split = int(len(X_train_full) * 0.875)
X_train, y_train = X_train_full[:split], y_train_full[:split]
X_val, y_val     = X_train_full[split:], y_train_full[split:]

# 6. PyTorch DataLoaders
batch_size = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def to_loader(X, y, shuffle=False):
    ds = TensorDataset(
        torch.tensor(X, dtype=torch.float32),
        torch.tensor(y, dtype=torch.float32).unsqueeze(1)
    )
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)

train_loader = to_loader(X_train, y_train, shuffle=True)
val_loader   = to_loader(X_val, y_val)
test_loader  = to_loader(X_test, y_test)

# 7. Weakened LSTM with Update Gate Only
class LSTM_UpdateOnly_Weak(nn.Module):
    def __init__(self, in_channels, hidden_dim=32, scale_factor=0.1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.scale_factor = scale_factor

        # Update gate
        self.W_z = nn.Linear(in_channels, hidden_dim)
        self.U_z = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.b_z = nn.Parameter(torch.zeros(hidden_dim))

        # Candidate state
        self.W_c = nn.Linear(in_channels, hidden_dim)
        self.U_c = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.b_c = nn.Parameter(torch.zeros(hidden_dim))

        # Simplified output head
        self.return_layer = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        batch_size, seq_length, _ = x.size()
        h = torch.zeros(batch_size, self.hidden_dim, device=x.device)
        c = torch.zeros(batch_size, self.hidden_dim, device=x.device)

        for t in range(seq_length):
            x_t = x[:, t, :]
            z_t = torch.sigmoid(self.W_z(x_t) + self.U_z(h) + self.b_z)
            candidate = torch.tanh(self.W_c(x_t) + self.U_c(h) + self.b_c)
            c = c + z_t * (candidate - c)
            h = torch.tanh(c)

        ret = torch.tanh(self.return_layer(h)) * self.scale_factor
        last_close = x[:, -1, 0].unsqueeze(1)
        return last_close * (1 + ret)

# 8. Training & Early Stopping
def train_model(model, train_loader, val_loader,
                lr=1e-3, epochs=50, patience=5):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    best_val = float('inf')
    best_state = None
    counter = 0

    for epoch in range(1, epochs + 1):
        model.train()
        train_losses = []
        for Xb, yb in train_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            optimizer.zero_grad()
            out = model(Xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        model.eval()
        val_losses = []
        with torch.no_grad():
            for Xb, yb in val_loader:
                Xb, yb = Xb.to(device), yb.to(device)
                val_losses.append(criterion(model(Xb), yb).item())

        avg_train = np.mean(train_losses)
        avg_val   = np.mean(val_losses)
        print(f'Epoch {epoch}: Train={avg_train:.6f}, Val={avg_val:.6f}')

        if avg_val < best_val:
            best_val = avg_val
            best_state = model.state_dict()
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print('Early stopping')
                break

    model.load_state_dict(best_state)
    return model

# 9. Evaluation
def evaluate(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for Xb, yb in loader:
            Xb = Xb.to(device)
            out = model(Xb).cpu().numpy()
            preds.append(out)
            trues.append(yb.numpy())
    preds = np.vstack(preds).flatten()
    trues = np.vstack(trues).flatten()

    def inv(x):
        tmp = np.zeros((len(x), data.shape[1]))
        tmp[:,0] = x
        return scaler.inverse_transform(tmp)[:,0]

    p_inv, t_inv = inv(preds), inv(trues)
    return {
        'R2': r2_score(t_inv, p_inv),
        'MAE': mean_absolute_error(t_inv, p_inv),
        'MSE': mean_squared_error(t_inv, p_inv),
        'RMSE': np.sqrt(mean_squared_error(t_inv, p_inv)),
        'MAPE': mean_absolute_percentage_error(t_inv, p_inv) * 100,
        'EVS': explained_variance_score(t_inv, p_inv)
    }, p_inv, t_inv

# 10. Run weakened model
model = LSTM_UpdateOnly_Weak(in_channels=data.shape[1]).to(device)
model = train_model(model, train_loader, val_loader)
metrics, preds, actuals = evaluate(model, test_loader)

print('Weakened Update Only LSTM Test Metrics:')
for k, v in metrics.items():
    print(f'  {k}: {v:.4f}')

# 11. Plot
plt.figure(figsize=(10, 4))
plt.plot(actuals, label='Actual')
plt.plot(preds, label='Predicted')
plt.legend()
plt.title('Weakened Update Only LSTM')
plt.show()

## LSTM + Modulation Component

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import (
    r2_score,
    mean_absolute_error,
    mean_squared_error,
    explained_variance_score,
    mean_absolute_percentage_error
)
import matplotlib.pyplot as plt

# 1. Seed for reproducibility
def set_seed(seed=30):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(28)

# Load or prepare your DataFrame 'data' here
# data = ...  # your preprocessed DataFrame with shape (n_samples, n_features)

# 2. Sequence creation
def create_sequences(arr, seq_length=20, target_idx=0):
    X, y = [], []
    for i in range(len(arr) - seq_length):
        X.append(arr[i: i + seq_length])
        y.append(arr[i + seq_length, target_idx])
    return np.array(X), np.array(y)

# 3. Train/Test split + scaling
values = data.values
train_size = int(len(values) * 0.8)
train_data = values[:train_size]
test_data  = values[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data)
test_scaled  = scaler.transform(test_data)

# 4. Create sequences
seq_length = 20
X_train_full, y_train_full = create_sequences(train_scaled, seq_length, target_idx=0)
X_test, y_test = create_sequences(test_scaled, seq_length, target_idx=0)

# 5. Further split train → train/val (87.5%/12.5%)
split = int(len(X_train_full) * 0.875)
X_train, y_train = X_train_full[:split], y_train_full[:split]
X_val, y_val     = X_train_full[split:], y_train_full[split:]

# 6. PyTorch DataLoaders
batch_size = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def to_loader(X, y, shuffle=False):
    ds = TensorDataset(
        torch.tensor(X, dtype=torch.float32),
        torch.tensor(y, dtype=torch.float32).unsqueeze(1)
    )
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)

train_loader = to_loader(X_train, y_train, shuffle=True)
val_loader   = to_loader(X_val, y_val)
test_loader  = to_loader(X_test, y_test)

# 7. Weakened LSTM with Modulation Gate Only
class LSTM_ModulationOnly_Weak(nn.Module):
    def __init__(self, in_channels, hidden_dim=32, scale_factor=0.1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.scale_factor = scale_factor

        # Modulation gate
        self.W_m = nn.Linear(in_channels, hidden_dim)
        self.U_m = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.b_m = nn.Parameter(torch.zeros(hidden_dim))

        # Candidate state
        self.W_c = nn.Linear(in_channels, hidden_dim)
        self.U_c = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.b_c = nn.Parameter(torch.zeros(hidden_dim))

        # Simplified output head
        self.return_layer = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        batch_size, seq_length, _ = x.size()
        h = torch.zeros(batch_size, self.hidden_dim, device=x.device)
        c = torch.zeros(batch_size, self.hidden_dim, device=x.device)

        for t in range(seq_length):
            x_t = x[:, t, :]
            m_t = torch.sigmoid(self.W_m(x_t) + self.U_m(h) + self.b_m)
            candidate = torch.tanh(self.W_c(x_t) + self.U_c(h) + self.b_c)
            c = m_t * c + (1 - m_t) * candidate
            h = torch.tanh(c)

        ret = torch.tanh(self.return_layer(h)) * self.scale_factor
        last_close = x[:, -1, 0].unsqueeze(1)
        return last_close * (1 + ret)

# 8. Training & Early Stopping
def train_model(model, train_loader, val_loader,
                lr=1e-3, epochs=50, patience=5):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    best_val = float('inf')
    best_state = None
    counter = 0

    for epoch in range(1, epochs + 1):
        model.train()
        train_losses = []
        for Xb, yb in train_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            optimizer.zero_grad()
            out = model(Xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        model.eval()
        val_losses = []
        with torch.no_grad():
            for Xb, yb in val_loader:
                Xb, yb = Xb.to(device), yb.to(device)
                val_losses.append(criterion(model(Xb), yb).item())

        avg_train = np.mean(train_losses)
        avg_val   = np.mean(val_losses)
        print(f'Epoch {epoch}: Train={avg_train:.6f}, Val={avg_val:.6f}')

        if avg_val < best_val:
            best_val = avg_val
            best_state = model.state_dict()
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print('Early stopping')
                break

    model.load_state_dict(best_state)
    return model

# 9. Evaluation
def evaluate(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for Xb, yb in loader:
            Xb = Xb.to(device)
            out = model(Xb).cpu().numpy()
            preds.append(out)
            trues.append(yb.numpy())
    preds = np.vstack(preds).flatten()
    trues = np.vstack(trues).flatten()

    def inv(x):
        tmp = np.zeros((len(x), data.shape[1]))
        tmp[:,0] = x
        return scaler.inverse_transform(tmp)[:,0]

    p_inv, t_inv = inv(preds), inv(trues)
    return {
        'R2': r2_score(t_inv, p_inv),
        'MAE': mean_absolute_error(t_inv, p_inv),
        'MSE': mean_squared_error(t_inv, p_inv),
        'RMSE': np.sqrt(mean_squared_error(t_inv, p_inv)),
        'MAPE': mean_absolute_percentage_error(t_inv, p_inv) * 100,
        'EVS': explained_variance_score(t_inv, p_inv)
    }, p_inv, t_inv

# 10. Run weakened model
model = LSTM_ModulationOnly_Weak(in_channels=data.shape[1]).to(device)
model = train_model(model, train_loader, val_loader)
metrics, preds, actuals = evaluate(model, test_loader)

print('Weakened Modulation Only LSTM Test Metrics:')
for k, v in metrics.items():
    print(f'  {k}: {v:.4f}')

# 11. Plot
plt.figure(figsize=(10, 4))
plt.plot(actuals, label='Actual')
plt.plot(preds, label='Predicted')
plt.legend()
plt.title('Weakened Modulation Only LSTM')
plt.show()


## Final Model (StockTrendNet)

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, explained_variance_score
from torch.utils.data import TensorDataset, DataLoader

# -------------------------------
# 1. Set Seeds for Reproducibility
# -------------------------------
def set_random_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_random_seed(30)

# -------------------------------
# 2. Load & Preprocess Data
# -------------------------------
# Make sure to load your DataFrame before this step.
# For example: data = pd.read_csv('your_data.csv')
df = data.copy()  # 'data' should be defined
all_cols = df.columns.tolist()

target = "Close"
# Ensure the target column is the first in the feature list.
feature_cols = [target] + [col for col in all_cols if col != target]
df = df[feature_cols].copy()

# Convert numeric columns (remove commas if needed)
for col in feature_cols:
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')
df = df.dropna()

# -------------------------------
# 3. Create Sequences from Time Series Data
# -------------------------------
def create_sequences(data, seq_length=20, target_idx=0):
    """
    Creates sequences from data.
    Each sequence of length 'seq_length' has a target as the value at index target_idx
    of the next timestep.
    """
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length, target_idx])  # target is "Close"
    return np.array(X), np.array(y)

data_values = df.values  # All features as inputs

# Split data into train (80%) and test (20%)
train_size = int(len(data_values) * 0.8)
train_data = data_values[:train_size]
test_data  = data_values[train_size:]

# Scale the data using MinMaxScaler (applied on all features)
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data)
test_scaled  = scaler.transform(test_data)

# Create sequences with a sequence length of 20
seq_length = 20
X_train, y_train = create_sequences(train_scaled, seq_length=seq_length, target_idx=0)
X_test, y_test   = create_sequences(test_scaled, seq_length=seq_length, target_idx=0)

# Further split training data into train and validation (e.g., 87.5% train, 12.5% validation)
train_split = int(len(X_train) * 0.875)
X_val = X_train[train_split:]
y_val = y_train[train_split:]
X_train = X_train[:train_split]
y_train = y_train[:train_split]

print("Train shape:", X_train.shape)
print("Validation shape:", X_val.shape)
print("Test shape:", X_test.shape)

# -------------------------------
# 4. Prepare PyTorch Datasets & DataLoaders
# -------------------------------
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_val_t   = torch.tensor(X_val, dtype=torch.float32)
y_val_t   = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)
X_test_t  = torch.tensor(X_test, dtype=torch.float32)
y_test_t  = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

batch_size = 32
train_dataset = TensorDataset(X_train_t, y_train_t)
val_dataset   = TensorDataset(X_val_t, y_val_t)
test_dataset  = TensorDataset(X_test_t, y_test_t)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# -------------------------------
# 5. Define Merged Custom LSTM Model
# -------------------------------
class CustomLSTMModel(nn.Module):
    def __init__(self, in_channels, seq_length=20, hidden_dim=64, dense_dim=32, scale_factor=0.2):
        """
        A custom LSTM model that internally uses its own LSTM cell logic.
        Processes the input sequence by iterating over every time step.
        """
        super(CustomLSTMModel, self).__init__()
        self.seq_length = seq_length
        self.scale_factor = scale_factor
        self.hidden_dim = hidden_dim
        
        # LSTM cell parameters (merged from CustomLSTMCell)
        self.W_z = nn.Linear(in_channels, hidden_dim)
        self.U_z = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.b_z = nn.Parameter(torch.zeros(hidden_dim))
        
        self.W_m = nn.Linear(in_channels, hidden_dim)
        self.U_m = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.b_m = nn.Parameter(torch.zeros(hidden_dim))
        
        self.W_c = nn.Linear(in_channels, hidden_dim)
        self.U_c = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.b_c = nn.Parameter(torch.zeros(hidden_dim))
        
        # Residual connection: project previous hidden state
        self.residual_linear = nn.Linear(hidden_dim, hidden_dim)
        self.layer_norm = nn.LayerNorm(hidden_dim)
        
        # Skip connection: project the input to hidden_dim
        self.input_linear = nn.Linear(in_channels, hidden_dim)
        
        # Dense layers for output processing
        self.dense = nn.Sequential(
            nn.Linear(hidden_dim, dense_dim),
            nn.ReLU()
        )
        self.return_layer = nn.Linear(dense_dim, 1)
    
    def forward(self, x):
        # x shape: (batch, seq_length, in_channels)
        batch_size = x.size(0)
        device = x.device
        
        # Initialize hidden and cell states to zeros
        h = torch.zeros(batch_size, self.hidden_dim, device=device)
        c = torch.zeros(batch_size, self.hidden_dim, device=device)
        
        # Process the sequence one timestep at a time
        for t in range(self.seq_length):
            x_t = x[:, t, :]
            z_t = torch.sigmoid(self.W_z(x_t) + self.U_z(h) + self.b_z)
            m_t = torch.sigmoid(self.W_m(x_t) + self.U_m(h) + self.b_m)
            candidate = torch.tanh(self.W_c(x_t) + self.U_c(h) + self.b_c)
            candidate = candidate + self.residual_linear(h)
            candidate = candidate + self.input_linear(x_t)
            candidate = self.layer_norm(candidate)
            
            c = (1 - z_t) * c + z_t * candidate
            h = m_t * torch.tanh(c)
        
        # Compute dense layers and output
        dense_out = self.dense(h)
        long_term_return = torch.tanh(self.return_layer(dense_out)) * self.scale_factor
        
        # Use the last day's "Close" price from the input sequence (assumes first feature is "Close")
        last_day = x[:, -1, 0].unsqueeze(1)
        predicted_price = last_day * (1 + long_term_return)
        return predicted_price

# -------------------------------
# 6. Training and Evaluation Pipeline
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
in_channels = len(feature_cols)
model = CustomLSTMModel(in_channels, seq_length=seq_length, hidden_dim=64, dense_dim=32, scale_factor=0.2).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
best_val_loss = float('inf')
patience = 10
counter = 0

for epoch in range(num_epochs):
    model.train()
    train_losses = []
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    
    model.eval()
    val_losses = []
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            val_losses.append(loss.item())
    
    avg_train_loss = np.mean(train_losses)
    avg_val_loss = np.mean(val_losses)
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.6f} - Val Loss: {avg_val_loss:.6f}")
    
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_state = model.state_dict()
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered!")
            break

model.load_state_dict(best_model_state)

# -------------------------------
# 7. Evaluation on Test Data
# -------------------------------
model.eval()
predictions = []
true_values = []
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
        preds = model(batch_X)
        predictions.append(preds.cpu().numpy())
        true_values.append(batch_y.cpu().numpy())

predictions = np.vstack(predictions)
true_values = np.vstack(true_values)

def inverse_transform(values):
    dummy = np.zeros((len(values), len(feature_cols)))
    dummy[:, 0] = values.flatten()
    return scaler.inverse_transform(dummy)[:, 0]

predictions_inv = inverse_transform(predictions)
true_values_inv = inverse_transform(true_values)

r2 = r2_score(true_values_inv, predictions_inv)
mae = mean_absolute_error(true_values_inv, predictions_inv)
mse = mean_squared_error(true_values_inv, predictions_inv)
rmse = np.sqrt(mse)
evs = explained_variance_score(true_values_inv, predictions_inv)
mape = mean_absolute_percentage_error(true_values_inv, predictions_inv) * 100

r2_scaled = r2_score(true_values, predictions)
mae_scaled = mean_absolute_error(true_values, predictions)
mse_scaled = mean_squared_error(true_values, predictions)
rmse_scaled = np.sqrt(mse_scaled)
evs_scaled = explained_variance_score(true_values, predictions)
mape_scaled = mean_absolute_percentage_error(true_values, predictions) * 100

print("📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}")
print(f"  MSE: {mse:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAPE: {mape:.2f}%")
print(f"  EVS: {evs:.4f}")

print("\n--- Scaled Metrics ---")
print(f"R² Score: {r2_scaled:.4f}")
print(f"MAE: {mae_scaled:.4f}")
print(f"MSE: {mse_scaled:.4f}")
print(f"RMSE: {rmse_scaled:.4f}")
print(f"Explained Variance Score: {evs_scaled:.4f}")
print(f"MAPE: {mape_scaled:.2f}%")

plt.figure(figsize=(10,5))
plt.plot(true_values_inv, label='Actual')
plt.plot(predictions_inv, label='Predicted')
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.legend()
plt.show()
